In [9]:
import logging
import couchdb
import json
import sys
sys.path.append('./sentiment analysis/release')
import sentiment_prediction as senti
sys.path.append('./topic analysis')
import scorer
# sys.path.append('../util')
# import MapRegion

In [10]:
'''
    Get the source db(the db created by harverster)
    and the destination db (store the result)
'''
def get_db():
    config = None
    server = None
    db = None
    with open("db_config.json") as cf:
        config = json.load(cf)
        try:
            db_server = couchdb.Server(config["Servers"][0])
            source_db_name = config["DB"][0]
            dest_db_name = config["DB"][1]
            if dest_db_name in db_server:
                logging.info("Database %s has existed, creating a new one." %(dest_db_name))
                db_server.delete(dest_db_name)
                dest_db = db_server.create(dest_db_name)
            else:
                logging.info("Database %s does not exist, creating a new one." %(dest_db_name))
                dest_db = db_server.create(dest_db_name)
            if source_db_name not in db_server:
                logging.error("Database %s does not exist, something went wrong." %(source_db_name))
                return
            source_db = db_server[source_db_name]
        except Exception as e:
            logging.error("error happened: %s" %e)
            sys.exit(2)
    return source_db,dest_db

In [15]:
'''
    This is the excution part, the analysis result will be
    stored into the dest db
'''
def analyze(source_db,dest_db,modelpath,pklpath):
    analysor = senti.sentianalysor(modelpath, pklpath, stemmer = False)
    topic_scorer = []
    alcohols_dict = "./topic analysis/dictionary/alcohols.txt"
    fastfood_dict = "./topic analysis/dictionary/fastfood.txt"
    smoking_dict = "./topic analysis/dictionary/smoking.txt"
    alcohols_scorer = scorer.Scorer(analysor,alcohols_dict)
    fastfood_scorer = scorer.Scorer(analysor,fastfood_dict)
    smoking_scorer = scorer.Scorer(analysor,alcohols_dict)
    count = 0
    result = []
    for ele in source_db:
        count += 1
        created_at = source_db[ele]["created_at"]
        raw_text = source_db[ele]["text"]
        location = source_db[ele]["location"]
        coordinates = source_db[ele]["coordinates"]
        label,proba = analysor.prediction(raw_text)
        new_text = analysor.preprocess_tweet(raw_text)
        #0:NEGATIVE 1:POSITIVE
        label_flag = 0 if label=="NEGATIVE" else 1
        #Some problems loading MapRegion
        region = None
        alcohols_sc = alcohols_scorer.get_score(new_text,"POSITIVE")
        fastfood_sc = fastfood_scorer.get_score(new_text,"POSITIVE")
        smoking_sc = smoking_scorer.get_score(new_text,"POSITIVE")
        result.append({"created_at":created_at,"text":new_text,"label":label_flag,"probability":proba,
                       "alcohols_score":alcohols_sc,"fastfood_score":fastfood_sc,"smoking_score":smoking_sc})
        if count == 100:
            dest_db.update(result)
            count = 0
            result = []
        print(count)
    if count != 0 and len(result) != 0:
        dest_db.update(result)
    print("Analyze successfully.")

In [27]:
if __name__ == "__main__":
    modelpath = './sentiment analysis/model/sentiment_lstm.h5'
    pklpath = './sentiment analysis/model/sentiment140-freqdist.pkl'
    source_db,dest_db = get_db()
    analyze(source_db,dest_db,modelpath,pklpath)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
Analyze successfully.


In [31]:
db_server = couchdb.Server("http://127.0.0.1:5984")

count = 0
dest_db = db_server["result"]
for ele in dest_db:
    if dest_db[ele]["alcohols_score"] >= 0.5:
        print(dest_db[ele])
count

<Document 'c339b3e7f2453a0ee28e0d76650d2e0b'@'1-5f96f63b5d70d84ccd574363b2bc9366' {'created_at': 'Mon Jan 01 02:09:26 +0000 2018', 'text': 'drinking a saison rose by at photo', 'label': 1, 'probability': 0.8765932321548462, 'alcohols_score': 1.727997557783723, 'fastfood_score': 0.0, 'smoking_score': 1.727997557783723}>
<Document 'c339b3e7f2453a0ee28e0d76650d6d79'@'1-3b4f636a52e6baf61e0bebf23e8abf7a' {'created_at': 'Mon Jan 01 02:23:39 +0000 2018', 'text': 'lovely fresh fruity session beer drinking an obsession by at', 'label': 1, 'probability': 0.9912427663803101, 'alcohols_score': 2.4454286156715153, 'fastfood_score': 0.0, 'smoking_score': 2.4454286156715153}>
<Document 'c339b3e7f2453a0ee28e0d76650e04fa'@'1-d55426d2e78db2d2ea5c8c785e12138b' {'created_at': 'Mon Jan 01 02:46:21 +0000 2018', 'text': 'drinking an obsession by at photo', 'label': 1, 'probability': 0.9943922162055969, 'alcohols_score': 1.9602105720785261, 'fastfood_score': 0.0, 'smoking_score': 1.9602105720785261}>
<Documen

0